# Metadata Preparation
This notebook carries out the semi-manual metadata cleaning steps that aim to correct any noticed errors remaining after the RiboSeq Metadata [R scripts](https://github.com/Roleren/riboseq_metadata)

In [1]:
import pandas as pd 


In [2]:
latest_metadata_path = "/home/jack/projects/Metadata/data/standardized_columns_final_2024-04-22.csv"

## Trips-Viz Metadata Matching

In [3]:
files = pd.read_csv('/home/jack/projects/RiboSeqOrg-DataPortal/data/trips_files.csv',
                    names=['file_id', 'organism_id', 'study_id', 'file_name', 'file_description', 'file_type', 'owner', 'mapped_reads', 'control', 'cell_line'])
studies = pd.read_csv('/home/jack/projects/RiboSeqOrg-DataPortal/data/trips_studies.csv',
                      names=['study_id', 'organism_id', 'study_name', 'paper_authors', 'srp_nos', 'paper_year', 'paper_pmid', 'paper_link', 'gse_nos', 'adapters', 'paper_title', 'description', 'private', 'owner'])
filtered = pd.read_csv(latest_metadata_path)

/tmp/ipykernel_2084577/757137606.py:5: DtypeWarning: Columns (30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,73,89,91,93,94,95,100,101,102,104,105,108,109,110,111,112,114,115) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered = pd.read_csv(latest_metadata_path)


In [4]:
files.shape

(12725, 10)

In [5]:
studies.head()

,study_id,organism_id,study_name,paper_authors,srp_nos,paper_year,paper_pmid,paper_link,gse_nos,adapters,paper_title,description,private,owner
NaN,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0.0,-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1.0,1,10,Nedialkova15,"Nedialkova DD, Leidel SA",SRP056647,2015,26052047,"""https://linkinghub.elsevier.com/retrieve/pii/...",GSE67387,NaN,Optimization of Codon Translation Rates via tR...,Ribosome profiling of wild-type and tRNA modif...,0,1
2.0,2,7,Dunn13,"Dunn JG, Foo CK, Belletier NG, Gavis ER, Weiss...",SRP028243,2013,24302569,"""https://doi.org/10.7554/eLife.01179""",GSE49197,CTGTAGGCACCATCAAT,Ribosome profiling reveals pervasive and regul...,12 samples of Drosophila ribosome profiling an...,0,1
3.0,3,8,Nakahigashi16,"Nakahigashi K, Takai Y, Kimura M, Abe N, Nakay...",DRP003075,2016,NaN,https://academic.oup.com/dnaresearch/article-l...,Nakahigashi16,NaN,Comprehensive identification of translation st...,NaN,0,1


In [6]:
public_studies = studies[studies['private'] == 0]
public_study_ids = list(public_studies['study_id'].values)

In [7]:
files = files[files['study_id'].isin(public_study_ids)]
files = files[files['file_type'] == 'riboseq']
files.shape

(2649, 10)

In [8]:
organisms = pd.read_csv('/home/jack/projects/RiboSeqOrg-DataPortal/data/trips_orgnaims.csv',
                        names=['organism_id', 'organism_name', 'transcriptome_list', 'gwips_databasename', 'gwips_clade', 'gwips_organism', 'gwips_database', 'default_transcript', 'private', 'owner'])
                        

In [9]:
organisms.head()

,organism_id,organism_name,transcriptome_list,gwips_databasename,gwips_clade,gwips_organism,gwips_database,default_transcript,private,owner
0,1,saccharomyces_cerevisiae,sgd,yeast,yeast,S.+cerevisiae,sacCer3,YPR122W,0,1
1,2,mycoplasma_hyorhinis,Ensembl_release37,mycoplasma_hyorhinis,mycoplasma_hyorhinis,M.+hyorinis,mh,AFX74150,1,1
2,3,mus_musculus,Gencode_M14,mouse,mammal,Mouse,mm10,ENSMUST00000037796,0,1
3,4,homo_sapiens,Gencode_v25,homo_sapiens,mammal,Human,hg38,ENST00000558401,0,1
4,5,homo_sapiens_polio,Ensembl_2011,homo_sapiens,mammal,Human,hg38,polio,1,1


In [10]:
with open('/home/jack/projects/RiboSeqOrg-DataPortal/data/trips_model.tsv', 'w') as f:
    for row in files.iterrows():
        match = filtered[filtered['Run'] == row[1]['file_name'].split('.')[0]]
        if len(match['Run'].values) == 0:
            run = 'NA'
            bioproject = 'NA'
        else:
            run = match['Run'].values[0]
            bioproject = match['BioProject'].values[0]
        organism = organisms[organisms['organism_id'] == row[1]['organism_id']]
        if organism.shape[0] == 0:
            continue
        study = studies[studies['study_id'] == row[1]['study_id']]
        f.write(f"{bioproject}\t{run}\t{row[1]['file_id']}\t{row[1]['file_name'].split('.')[0]}\t{study['study_name'].values[0]}\t{study['srp_nos'].values[0]}\t{study['gse_nos'].values[0]}\t{study['paper_pmid'].values[0]}\t{organism['organism_name'].values[0]}\t{organism['transcriptome_list'].values[0]}\n")

## Metadata Cleanup

In [11]:
latest_metadata = pd.read_csv(latest_metadata_path)

/tmp/ipykernel_2084577/4029497756.py:1: DtypeWarning: Columns (30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,73,89,91,93,94,95,100,101,102,104,105,108,109,110,111,112,114,115) have mixed types. Specify dtype option on import or set low_memory=False.
  latest_metadata = pd.read_csv(latest_metadata_path)


In [12]:
# List of column pairs to update
column_pairs = [('TISSUE', 'TISSUE_st'), ('CELL_LINE', 'CELL_LINE_st'), ('INHIBITOR', 'INHIBITOR_st'),
                ('CONDITION', 'CONDITION_st'), ('REPLICATE', 'REPLICATE_st'), ('LIBRARYTYPE', 'LIBRARYTYPE_st'),
                ('FRACTION', 'FRACTION_st'), ('TIMEPOINT', 'TIMEPOINT_st'), ('ScientificName', 'scientific_name')]

for main_col, source_col in column_pairs:
    # Update the main column with non-NaN values from the source column
    if source_col in latest_metadata.columns:
        latest_metadata.loc[latest_metadata[source_col].notnull(), main_col] = latest_metadata.loc[latest_metadata[source_col].notnull(), source_col]

# then drop the source columns
latest_metadata.drop(columns=[col for _, col in column_pairs if col in latest_metadata.columns], inplace=True)

In [13]:
import numpy as np

In [14]:
latest_metadata.loc[latest_metadata['Study_Pubmed_id'] == 1, 'Study_Pubmed_id'] = np.nan
latest_metadata.loc[latest_metadata['AUTHOR'] == 'Makar', 'AUTHOR'] = np.nan
latest_metadata['AUTHOR'].value_counts()


AUTHOR
Atger          884
Bazzini        526
Marcel         200
Chou           165
Mohan          158
              ... 
Latif            2
Gupta            1
Nakahigashi      1
Hanson           1
Weinberg         1
Name: count, Length: 479, dtype: int64

In [15]:
nan_percentages = latest_metadata.isna().mean() * 100

#print any that are not 0% na 
nan_percentages[nan_percentages > 0]


avgLength                  0.390836
LibraryName               67.331536
Study_Pubmed_id           31.334232
CenterName                 0.936658
AUTHOR                    29.083558
                            ...    
Barcode                   98.659030
Monosome_purification    100.000000
Nuclease                  99.501348
Kit                       99.258760
Info                       4.865229
Length: 92, dtype: float64

In [16]:
len(latest_metadata['CELL_LINE'].unique())

477

In [17]:
# replace anything in the CELL_LINE column that with lymphoblastoid in cell line 
# There were many with GMXXXXlymphoblasoid cell line as the cell line 
# Create a boolean mask without NaN/NA values
mask = latest_metadata['CELL_LINE'].str.contains('lymphoblastoid', case=False, na=False)

# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = 'lymphoblastoid'

In [64]:
latest_metadata['CELL_LINE'].value_counts()

CELL_LINE
HEK293                      861
3T3                         229
A549                        183
mESC                        155
E14                         150
                           ... 
MOLM13leukemia cell line      1
HL60leukemia cell line        1
HBE                           1
GM19238                       1
GM19204                       1
Name: count, Length: 310, dtype: int64

In [18]:
len(latest_metadata.columns)

120

In [19]:
latest_metadata.loc[latest_metadata['translation inhibitor'] == "8 µg/mL harringtonine for 10 min, then 0.1 mg/mL cycloheximide in lysis buffer", 'translation inhibitor'] = 'harr'

In [20]:
# drop columns that are not helpful
unwanted = [ 
    'Run.1',
    'BioProject.1',
    'name',
    'not_unique',
    'translation inhibitor',
    'mouse line',
    'type',
    'disease state',
    'diagnosis',
    'cell status',
    'isolates',
    'lncrna probes',
    'sequencing type',
    'knockdown or knockout',
    'input',
    'fragmentation',
    'lentivirus',
    'animal group',
    'cell stage',
    'biol replicate',
    'tech replicate',
    'tretment',
    'progenitor cell type',
    'geographic location (country and/or sea)',
    'specimen with known storage state',
    'rnasei treatment',
    'model',
    'identity',
    'cdna type',
    'primer set',
    'lentivirally transduced transgenes',
    'genotype/variaion',
    ]

drop_list = [col for col in latest_metadata.columns if col in unwanted]

for col in latest_metadata.columns:
    if "Experimental" in col:
        drop_list.append(col)
latest_metadata.drop(columns=drop_list, inplace=True)

In [21]:
len(latest_metadata.columns)

73

In [22]:
latest_metadata.to_csv("../data/standardised_22-04-24.csv", index=False)

In [23]:
high_priority_columns = [
    'Run',
    'BioProject',
    'Study_Pubmed_id',
    'AUTHOR',
    'TISSUE',
    'CELL_LINE',
    'INHIBITOR',
    'CONDITION',
    'REPLICATE',
    'LIBRARYTYPE',
    'FRACTION',
    'TIMEPOINT',
    'ScientificName',
    'Sex',
]

In [24]:
combined = latest_metadata['CELL_LINE'] + "-" + latest_metadata['TISSUE'] + '-' + latest_metadata['ScientificName']
counts = combined.value_counts()
counts.to_csv('../data/counts.tsv', sep='\t', header=False)

## Specific Corrections

S2, a Drosophila embryonic cell line is often misidentified when Sample 2 is meant. As a result all non drosophila S2 cell line entries can be overwritten

In [25]:
import numpy as np

In [26]:
# Create a boolean mask without NaN/NA values where cell line is "S2" and organism is not Drosophila melanogaster
mask = (latest_metadata['CELL_LINE'] == 'S2') & (latest_metadata['ScientificName'] != 'Drosophila melanogaster')

# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = np.nan
latest_metadata.loc[mask, 'TISSUE'] = np.nan



In [27]:
mask = latest_metadata['CELL_LINE'] == "TSC2"

latest_metadata.loc[mask, 'CELL_LINE'] = np.nan
latest_metadata.loc[mask, 'TISSUE'] = np.nan


In [28]:
# Create a boolean mask without NaN/NA values where cell line is "S2" and organism is not Drosophila melanogaster
mask = (latest_metadata['CELL_LINE'] == 'H1') & (latest_metadata['ScientificName'] == 'Escherichia coli')

# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = np.nan
latest_metadata.loc[mask, 'TISSUE'] = np.nan

In [29]:
mask = (latest_metadata['CELL_LINE'] == 'PC3') & (latest_metadata['ScientificName'] == 'Neurospora crassa')

# latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]
# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = np.nan
latest_metadata.loc[mask, 'TISSUE'] = np.nan

In [30]:
mask = (latest_metadata['CELL_LINE'] == 'H1') & (latest_metadata['TISSUE'] == 'embryo') & (latest_metadata['ScientificName'] != 'Homo sapiens')

latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]
# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = np.nan
latest_metadata.loc[mask, 'TISSUE'] = np.nan


In [31]:
mask = (latest_metadata['TISSUE'] == 'stem') & (latest_metadata['ScientificName'] == 'Mus musculus')

latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]
# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = np.nan
latest_metadata.loc[mask, 'TISSUE'] = np.nan

In [32]:
mask = (latest_metadata['TISSUE'] == 'stem') & (latest_metadata['ScientificName'] == 'Homo sapiens')

latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]
# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = np.nan
latest_metadata.loc[mask, 'TISSUE'] = np.nan

In [33]:
mask = (latest_metadata['TISSUE'] == 'leaf') & (latest_metadata['CELL_LINE'] == 'H9')

latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]
# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = np.nan
latest_metadata.loc[mask, 'TISSUE'] = np.nan

In [34]:
mask = (latest_metadata['TISSUE'] != 'kidney') & (latest_metadata['CELL_LINE'] == 'HEK293')

latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]
# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = np.nan
latest_metadata.loc[mask, 'TISSUE'] = np.nan

In [35]:
mask = (latest_metadata['TISSUE'] != 'cervix') & (latest_metadata['CELL_LINE'] == 'HeLa')

latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]
# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = np.nan
latest_metadata.loc[mask, 'TISSUE'] = np.nan

In [36]:
mask = (latest_metadata['TISSUE'] == 'EBV-transformed lymphoblastoid cells') 

latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]
# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = np.nan

# latest_metadata.loc[mask, 'TISSUE'] = np.nan

In [37]:
mask = (latest_metadata['CELL_LINE'] == 'HEK293') & (latest_metadata['TISSUE'] == 'kidney') & (latest_metadata['ScientificName'] != 'Homo sapiens') 

latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]
# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = np.nan

latest_metadata.loc[mask, 'TISSUE'] = np.nan

In [38]:
mask = (latest_metadata['CELL_LINE'].str.endswith("-cell")) & (latest_metadata['TISSUE'] == "embryo")
mask = mask.fillna(False)
latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]
# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = np.nan
latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]


,Run,CELL_LINE,TISSUE,ScientificName
9708,SRR18113903,NaN,embryo,Homo sapiens
9709,SRR18113904,NaN,embryo,Homo sapiens
9710,SRR18113900,NaN,embryo,Homo sapiens
9711,SRR18113901,NaN,embryo,Homo sapiens
9712,SRR18113897,NaN,embryo,Homo sapiens
9713,SRR18113898,NaN,embryo,Homo sapiens
9725,SRR18113797,NaN,embryo,Homo sapiens
9726,SRR18113798,NaN,embryo,Homo sapiens
9727,SRR18113795,NaN,embryo,Homo sapiens
9728,SRR18113796,NaN,embryo,Homo sapiens


In [39]:
mask = (latest_metadata['CELL_LINE'].str.endswith("derived tumor")) & (latest_metadata['TISSUE'] == "Glioblastoma")
mask = mask.fillna(False)
latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]
# Update the 'CELL_LINE' column for rows where the mask is True
latest_metadata.loc[mask, 'CELL_LINE'] = np.nan
latest_metadata.loc[mask, 'TISSUE'] = np.nan

In [40]:
mask = (latest_metadata['CELL_LINE'] == "HCT116") & (latest_metadata['TISSUE'] != "colon")
mask = mask.fillna(False)
latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]
# Update the 'CELL_LINE' column for rows where the mask is True
# latest_metadata.loc[mask, 'CELL_LINE'] = np.nan
# latest_metadata.loc[mask, 'TISSUE'] = np.nan

,Run,CELL_LINE,TISSUE,ScientificName
8208,SRR13679796,HCT116,oocyte,Homo sapiens
8209,SRR13679797,HCT116,oocyte,Homo sapiens
8224,SRR13679812,HCT116,oocyte,Homo sapiens
8225,SRR13679813,HCT116,oocyte,Homo sapiens
8226,SRR13679814,HCT116,oocyte,Homo sapiens
8227,SRR13679815,HCT116,oocyte,Homo sapiens


In [41]:
latest_metadata.to_csv("../data/standardised_22-04-24_updated.csv", index=False)

In [42]:
combined = latest_metadata['CELL_LINE'] + "-" + latest_metadata['TISSUE'] + '-' + latest_metadata['ScientificName']
counts = combined.value_counts()
counts.to_csv('../data/counts.tsv', sep='\t', header=False)

In [43]:
mask = (latest_metadata['TISSUE'] == "embryo") & (latest_metadata['ScientificName'] == "Mus musculus") & (latest_metadata['CELL_LINE'] == np.nan)
mask = mask.fillna(False)
latest_metadata[mask][['Run', 'CELL_LINE', 'TISSUE', 'ScientificName']]

,Run,CELL_LINE,TISSUE,ScientificName


In [44]:
# convert all INHIBITOR to lowercase
latest_metadata['INHIBITOR'] = latest_metadata['INHIBITOR'].str.lower()

In [45]:
mask = (latest_metadata['INHIBITOR'] == "no treatment") | (latest_metadata['INHIBITOR'] == "none") | (latest_metadata['INHIBITOR'] == "no erythromycin")
latest_metadata.loc[mask, 'INHIBITOR'] = "untreated"

In [46]:
mask = (latest_metadata['INHIBITOR'].str.endswith("thapsigargin"))
mask = mask.fillna(False)
latest_metadata.loc[mask, 'INHIBITOR'] = "thapsigargin"

/tmp/ipykernel_2084577/3323106336.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask = mask.fillna(False)


In [47]:
# Make all INHIBITOR entries that are not 'chx', 'harr', ''lactim' np.nan
accepted_list = [
    'untreated',
    'chx',
    'harr',
    'lactim',
    'chx_harr',
    'chx_lactim',
    'frozen',
    'tetracycline',
    'thapsigargin',
    'anisomycin',
    'tunicamycin',
]
mask = ~(latest_metadata['INHIBITOR'].isin(accepted_list) | latest_metadata['INHIBITOR'].str.endswith("in"))
mask = mask.fillna(False)

latest_metadata.loc[mask, 'INHIBITOR'] = np.nan

In [48]:
mask = latest_metadata['INHIBITOR'].str.endswith("min")
mask = mask.fillna(False)

latest_metadata.loc[mask, 'INHIBITOR'] = np.nan


/tmp/ipykernel_2084577/341228739.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask = mask.fillna(False)


In [49]:
latest_metadata['INHIBITOR'].value_counts().to_csv('../data/inhib_counts.tsv', sep='\t', header=False)

In [50]:
latest_metadata.to_csv("../data/standardised_22-04-24_updated.csv", index=False)

In [51]:
mask = latest_metadata['LIBRARYTYPE'].str.startswith("Ribosome")
mask = mask.fillna(False)
latest_metadata.loc[mask, 'LIBRARYTYPE'] = "RFP"

/tmp/ipykernel_2084577/653559186.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask = mask.fillna(False)


In [52]:
mask = latest_metadata['LIBRARYTYPE'].str.contains("ibosome", case=False)
mask = mask.fillna(False)
latest_metadata.loc[mask, 'LIBRARYTYPE'] = "RFP"

/tmp/ipykernel_2084577/3772127391.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask = mask.fillna(False)


In [53]:
mask = latest_metadata['LIBRARYTYPE'].str.startswith("40S")
mask = mask.fillna(False)
latest_metadata.loc[mask, 'LIBRARYTYPE'] = "SSU"

/tmp/ipykernel_2084577/2833749186.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask = mask.fillna(False)


In [54]:
mask = latest_metadata['LIBRARYTYPE'].str.startswith("small ribosomal subunit")
mask = mask.fillna(False)
latest_metadata.loc[mask, 'LIBRARYTYPE'] = "SSU"

/tmp/ipykernel_2084577/1441698002.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask = mask.fillna(False)


In [55]:
mask = latest_metadata['LIBRARYTYPE'].str.startswith("80S")
mask = mask.fillna(False)
latest_metadata.loc[mask, 'LIBRARYTYPE'] = "LSU"

/tmp/ipykernel_2084577/3750572317.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask = mask.fillna(False)


In [56]:
mask = latest_metadata['LIBRARYTYPE'].str.startswith("large ribosomal subunit ")
mask = mask.fillna(False)
latest_metadata.loc[mask, 'LIBRARYTYPE'] = "LSU"

/tmp/ipykernel_2084577/3795245585.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask = mask.fillna(False)


In [57]:
mask = latest_metadata['LIBRARYTYPE'].str.startswith("Ribotag")
mask = mask.fillna(False)
latest_metadata.loc[mask, 'LIBRARYTYPE'] = "RiboTag"

/tmp/ipykernel_2084577/1738090903.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask = mask.fillna(False)


In [58]:
latest_metadata['LIBRARYTYPE'].value_counts().to_csv('../data/libtype_counts.tsv', sep='\t', header=False)

In [59]:
latest_metadata.to_csv("../data/standardised_22-04-24_updated.csv", index=False)

In [60]:
organism_list = [
    "Salmonella enterica",
    "Escherichia coli",
    "Saccharomyces cerevisiae",
    "Zymomonas mobilis",
    "Oryza sativa",
    "Streptomyces avermitilis",
    "Mycobacterium tuberculosis",
    "Streptomyces tsukubensis",
    "Staphylococcus aureus",
    "Trypanosoma cruzi",
    "Lacticaseibacillus rhamnosus",
    "Bacillus subtilis",
    "Caulobacter vibrioides",
    "Pseudomonas aeruginosa",
    "Mycobacteroides abscessus",
    "Schizosaccharomyces pombe",
    "Mycoplasmoides gallisepticum",
    "Plasmodium falciparum",
    "Streptomyces coelicolor",
    "Flavobacterium johnsoniae",
    "Mycoplasma pneumoniae",
    "Cryptococcus neoformans",
    "Mycolicibacterium smegmatis",
    "Sinorhizobium meliloti",
    "Bacteroides thetaiotaomicron",
    "Vibrio natriegens",
    "Vibrio vulnificus"
]

for organism in organism_list:
    mask = latest_metadata['ScientificName'].str.startswith(organism)
    mask = mask.fillna(False)
    latest_metadata.loc[mask, 'ScientificName'] = organism


In [61]:
mask = latest_metadata['ScientificName'].str.startswith("Severe acute respiratory syndrome coronavirus 2")
mask = mask.fillna(False)
latest_metadata.loc[mask, 'ScientificName'] = "SARS-CoV2"

In [62]:
latest_metadata['ScientificName'].value_counts().to_csv('../data/ScientificName_counts.tsv', sep='\t', header=False)

In [63]:
latest_metadata.to_csv("../data/standardised_22-04-24_updated.csv", index=False)